# Image format conversion integrity testing

In [1]:
from modules.recorder import recorder, write_video
import numpy as np, pandas as pd, cv2
import matplotlib.pyplot as plt

C:\Users\luisr\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


### Functions to calculate pixel difference

#### Pixel difference statistcs from two 3D image arrays

In [2]:
def diff_stats(frame1, frame2):

    isdiff = frame1 != frame2
    diff1, diff2 = frame1[isdiff], frame2[isdiff]
    diff_diff = np.abs(diff1.astype(int) - diff2.astype(int))

    return {
        'Altered pixels percentage': isdiff.mean(),
        'Altered pixels diff. average': diff_diff.mean(),
        'Altered pixels diff. standard deviation': diff_diff.std(),    
    }

#### Pixel difference statistcs from list of pairs of 3D image arrays

In [ ]:
def frames_diff_stats(frames1, frames2):
    df_diff = []
    for i, (frame1, frame2) in enumerate(zip(frames1, frames2)):
        df_diff.append(diff_stats(frame1, frame2))
    return pd.DataFrame(df_diff)

---
## Conversion integrity testing

#### array -> jpg -> array

In [8]:
folder = 'test'

rec = recorder(folder, n_frames=100, skip_method='histogram', skip_max=25, saveas='video')

frames = rec.capture('313') # get a few frames
frames_arr = [f for _, f in frames]

frames_jpg = []
for i, (filename, frame_arr) in enumerate(frames):
    filepath = f'{folder}/{filename}.jpg'
    cv2.imwrite(filepath, frame_arr)
    frame_jpg = cv2.imread(filepath)
    frames_jpg.append(frame_jpg)
    
df_diff = frames_diff_stats(frames_arr, frames_jpg)
display(df_diff.mean().rename('Average'))

Altered pixels percentage                  0.519092
Altered pixels diff. average               1.803515
Altered pixels diff. standard deviation    3.700593
Name: Average, dtype: float64

#### array -> mp4 -> array

In [11]:
rec = recorder(folder='test', n_frames=100,  skip_method='histogram', skip_max=25, saveas='video')

folder = 'test'

frames = rec.capture('313')
frames_arr = [f[1] for f in frames]

write_video(frames_arr, f'{folder}/313.mp4')

rec.url = folder + '/{}.mp4'
frames = rec.capture('313') # Capture video from file in path rec.url.format(code)
frames_mp4 = [f[1] for f in frames]

df_diff = []
for i, (frame_arr, frame_mp4) in enumerate(zip(frames_arr, frames_mp4)):
    df_diff.append(diff_stats(frame_arr, frame_mp4))
    
df_diff_mp4 = pd.DataFrame(df_diff)
display(df_diff_mp4.mean().rename('Average'))

Altered pixels percentage                  0.886486
Altered pixels diff. average               3.077592
Altered pixels diff. standard deviation    2.284948
Name: Average, dtype: float64